In [35]:
import numpy as np
import rasterio
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

# reading in geotiff file as numpy array
def read_tif(file: Path):

    if not file.exists():
        raise FileNotFoundError(f'File {file} not found')

    with rasterio.open(file) as dataset:
        arr = dataset.read()  # (bands X height X width)
        transform = dataset.transform
        crs = dataset.crs

    return arr.transpose((1, 2, 0)), transform, crs


# writing an array to a geo tiff file
def write_tif(file: Path, arr, transform, crs):

    if not file.parent.exists():
        file.parent.mkdir()

    height, width, bands = arr.shape
    with rasterio.open(
            file,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=bands,
            dtype=arr.dtype,
            crs=crs,
            transform=transform,
    ) as dst:
        for i in range(bands):
            dst.write(arr[:, :, i], i + 1)


SEED = 7

In [39]:
year = 2020
nobuilding = False
noglcm = True

data_folder = Path('C:/Users/shafner/slum_extent_mapping/land_cover_classification_v3/data')
pred_folder = f'predictions_nobuilding_{year}' if nobuilding else f'predictions_{year}'
pred_folder = data_folder / pred_folder



features_fname = f'features_Pan{year}_nobuilding.npy' if nobuilding else f'features_Pan{year}.npy'
features_file = data_folder / features_fname
features = np.load(str(features_file))
if noglcm:
    features = features[:, 0:26]
print(features.shape)

labels_fname = data_folder / f'labels_Pan{year}_nobuilding.npy' if nobuilding else f'labels_Pan{year}.npy'
labels_file = data_folder / labels_fname
labels = np.load(str(labels_file))
print(labels.shape)

print(np.unique(labels))
print(np.sum(np.isnan(features)))

# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=SEED)
rf_classifier = RandomForestClassifier(n_estimators=500, bootstrap=True, verbose=1, random_state=SEED)
# trained_rf_classifier = rf_classifier.fit(X_train, y_train)
# rf_score = trained_rf_classifier.score(X_test, y_test)
# print(rf_score)

(89292, 26)
(89292, 1)
[0 1 2 3 4 5]
0


In [40]:
trained_rf_classifier = rf_classifier.fit(features, labels)

<ipython-input-40-bfa5b735eab7>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  trained_rf_classifier = rf_classifier.fit(features, labels)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  3.4min finished


In [41]:
def classify_img(img_file: Path, trained_classifier):
    img, transform, crs = read_tif(img_file)
    if noglcm:
        img = img[:, :, 0:26]
    m, n, n_features = img.shape
    img_vector = np.reshape(img, (m * n, n_features))
    img_vector = np.nan_to_num(img_vector)
    pred_vector = trained_classifier.predict(img_vector)
    pred = np.reshape(pred_vector, (m, n))
    
    fname = f'pred_nobuilding_{img_file.stem}.tif' if nobuilding else f'pred_{img_file.stem}.tif'
    output_file = pred_folder / fname
    write_tif(output_file, pred[:, :, None], transform, crs)


tiles_folder = data_folder / f'sentinel2_{year}'
tile_files = [f for f in tiles_folder.glob('**/*')]
for tile_file in tqdm_notebook(tile_files):
    classify_img(tile_file, trained_rf_classifier)

<ipython-input-41-35b0ffa49fc2>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tile_file in tqdm_notebook(tile_files):


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   28.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    8.2s finished


In [45]:
year = 2016
nobuilding = False
data_folder = Path('C:/Users/shafner/slum_extent_mapping/land_cover_classification_v3/data')
pred_folder = f'predictions_nobuilding_{year}' if nobuilding else f'predictions_{year}'
pred_folder = data_folder / pred_folder


def get_coords(file: Path):
    file_parts = file.stem.split('-')
    coords = [int(file_parts[-2]), int(file_parts[-1])]
    return coords


def id2yx(patch_id: str) -> tuple:
    y, x = patch_id.split('-')
    return int(y), int(x)


def combine_tif_patches(folder: Path, basename: str, delete_tiles: bool = False, dtype=np.int8):
    files = [f for f in folder.glob('**/*')]
    coords = [get_coords(f) for f in files]
    print(coords)
    i_coords = [coord[0] for coord in coords]
    j_coords = [coord[1] for coord in coords]

    max_i = max(i_coords)
    max_j = max(j_coords)

    ul_file = folder / f'{basename}-{0:010d}-{0:010d}.tif'
    ul_arr, transform, crs = read_tif(ul_file)
    tile_height, tile_width, n_bands = ul_arr.shape
    assert (tile_height == tile_width)
    tile_size = tile_height

    lr_file = folder / f'{basename}-{max_i:010d}-{max_j:010d}.tif'
    lr_arr, _, _ = read_tif(lr_file)
    lr_height, lr_width, _ = lr_arr.shape

    mosaic_height = max_i + lr_height
    mosaic_width = max_j + lr_width
    mosaic = np.full((mosaic_height, mosaic_width, n_bands), fill_value=-1, dtype=dtype)

    for index, file in enumerate(files):
        tile, _, _ = read_tif(file)
        i_start, j_start = get_coords(file)
        i_end = i_start + tile_size
        j_end = j_start + tile_size
        mosaic[i_start:i_end, j_start:j_end, ] = tile
        if delete_tiles:
            file.unlink()

    output_file = folder / f'{basename}.tif'
    write_tif(output_file, mosaic, transform, crs)

output_name = f'pred_nobuilding_s2_Pan{year}' if nobuilding else f'pred_s2_Pan{year}'
combine_tif_patches(pred_folder, output_name)


[[0, 0], [0, 1024], [0, 2048], [0, 3072], [1024, 0], [1024, 1024], [1024, 2048], [1024, 3072], [2048, 0], [2048, 1024], [2048, 2048], [2048, 3072], [3072, 0], [3072, 1024], [3072, 2048], [3072, 3072]]


In [47]:
year = 2020

pred_file = data_folder / f'predictions_nobuilding_{year}' / f'pred_nobuilding_s2_Pan{year}.tif'
imp_file = data_folder / 'soft_rmse_adamw' / f'prob_sentinel2_{year}_soft_rmse_adamw.tif'

pred, geotransform, crs = read_tif(pred_file)
imp, _, _ = read_tif(imp_file)

m, n, _ = pred.shape
imp = imp[:m, :n, 0]

THRESHOLD = 33

is_impervious = np.squeeze(imp > THRESHOLD)
print(pred.shape, is_impervious.shape)
pred[is_impervious, ] = 1

output_file = data_folder / f'hybrid_pred_{year}.tif'
write_tif(output_file, pred, geotransform, crs)


(3380, 3390, 1) (3380, 3390)
